# AB Test를 위한 Python Script
 ## Read Test data > 샘플별로 서버에 send > 서버 response 받고, feedback 전송

In [10]:
import json # will be needed for saving preprocssing details
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import requests

In [11]:
# load datatset
df = pd.read_csv('https://raw.githubusercontent.com/pplonski/datasets-for-start/master/adult/data.csv', skipinitialspace=True)
x_cols = [c for c in df.columns if c != 'income']

# set input matrix and target column
X = df[x_cols]
y = df['income']

#show first 5 rows of data
df.head()

age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  capital-loss  hours-per-week native-country income  
0          2174             0              40  United-States  <=50K  
1             0             0              13  United-States  <=50K  
2             0             0              40  United-States  <=50K  
3             0             0              40  United-States  <=50K  
4             0             0              40           Cuba  <=50K

In [12]:
# data split train/ test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 1234)


In [14]:
# 처음 100행을 A/B Test 용 데이터로 사용
for i in range(100):
    input_data = dict(X_test.iloc[i])
    target = y_test.iloc[i]
    r= requests.post("http://127.0.0.1:8000/api/v1/income_classifier/predict?status=ab_testing", input_data)
    response = r.json()
    
    #provide feedback
    requests.put("http://127.0.0.1:8000/api/v1/mlrequests/{}".format(response["request_id"]), {"feedback":target})